<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

## Automatic Test Generation

In this project, we will build an automatic test generation and grading platform!
All we have to do is to provide a topic, the number of questions and the number of options for each question!
Based on this information, a suitable test is generated, presented to the user and graded automatically!

## Imports

In [8]:
import os
import openai


## OpenAI API

Remember to use the notebook as shown, you must set your OpenAI API Key as an environment variable. Obviously, there are many ways you could provide your API Key to the Python code, input() or even hard-coded, but those are typically not recommended for safety reasons. Having it as an environment variable let's the key live on the computer, but not actually be present in the code.

### Set-up Open AI API Key

We'll only need to do this once per computer

In [14]:
# Uncomment below and swap in your key to place your environment key using Python
# Then you can delete the key string and the code cell below will still work!
# os.environ["OPENAI_API_KEY"] = "your key goes here", comment out after invoking this line

In [15]:
openai.api_key = os.getenv("OPENAI_API_KEY")

### Tell GPT how to generate the test

We tell GPT to create a multiple choiz quiz. Hence we define the topic, the number of possible answers as well as the number of questions.
To enable automatical grading later, GPT needs to incorporate the correct answer!


In [16]:
def create_test_prompt(topic, num_questions, num_possible_answers):
    prompt = f"Create a multiple choice quiz on the topic of {topic} consisting of {num_questions} questions. " \
                 + f"Each question should have {num_possible_answers} options. "\
                 + f"Also include the correct answer for each question using the starting string 'Correct Answer: '."
    return prompt

In [17]:
create_test_prompt("Python", 4, 4)

"Create a multiple choice quiz on the topic of Python consisting of 4 questions. Each question should have 4 options. Also include the correct answer for each question using the starting string 'Correct Answer: '."

### OpenAI API Call
Let's use text-davinci-003 for normal text generation

In [18]:
response = openai.Completion.create(engine="text-davinci-003",
                                            prompt=create_test_prompt("Python", 4, 4),
                                            max_tokens=256,
                                            temperature=0.7)

In [19]:
response

<OpenAIObject text_completion id=cmpl-7jC1WSAnKRQMni6ztq9lKKlxvTELl at 0x11511dd90> JSON: {
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "id": "cmpl-7jC1WSAnKRQMni6ztq9lKKlxvTELl",
  "object": "text_completion",
  "created": 1691005334,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\n\nQuestion 1: What is the main programming language used for the Raspberry Pi?\nA) Java \nB) Python \nC) HTML \nD) C++ \nCorrect Answer: B) Python\n\nQuestion 2: What is the correct syntax for a for loop in Python?\nA) for (i=0; i<x; i++)\nB) for i in range(x):\nC) for (i=0; i>x; i++)\nD) for i in x:\nCorrect Answer: B) for i in range(x):\n\nQuestion 3: What is the output of the following code in Python?\nmy_list = [1, 2, 3]\nmy_list.append(4)\n\nA) [1, 2, 3, 4]\nB) [4, 2, 3, 1]\nC) 4\nD) None\nCorrect Answer: A) [1, 2, 3, 4]\n\nQuestion 4: What is the proper s

In [20]:
response["choices"][0]["text"]

'\n\nQuestion 1: What is the main programming language used for the Raspberry Pi?\nA) Java \nB) Python \nC) HTML \nD) C++ \nCorrect Answer: B) Python\n\nQuestion 2: What is the correct syntax for a for loop in Python?\nA) for (i=0; i<x; i++)\nB) for i in range(x):\nC) for (i=0; i>x; i++)\nD) for i in x:\nCorrect Answer: B) for i in range(x):\n\nQuestion 3: What is the output of the following code in Python?\nmy_list = [1, 2, 3]\nmy_list.append(4)\n\nA) [1, 2, 3, 4]\nB) [4, 2, 3, 1]\nC) 4\nD) None\nCorrect Answer: A) [1, 2, 3, 4]\n\nQuestion 4: What is the proper syntax for declaring a function in Python?\nA) function funcName()\nB) def funcName()\nC) funcName()\nD) funcName = ()\nCorrect Answer:'

### Q/A Extraction

We now need to extract the questions and answers to present them to the students later

In [21]:
def create_student_view(test, num_questions):
    student_view = {1 : ""}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer:"):
            student_view[question_number] += line+"\n"
        else:

            if question_number < num_questions:
                question_number+=1
                student_view[question_number] = ""
    return student_view
 

In [22]:
create_student_view(response["choices"][0]["text"], 4)

{1: '\n\nQuestion 1: What is the main programming language used for the Raspberry Pi?\nA) Java \nB) Python \nC) HTML \nD) C++ \n',
 2: '\nQuestion 2: What is the correct syntax for a for loop in Python?\nA) for (i=0; i<x; i++)\nB) for i in range(x):\nC) for (i=0; i>x; i++)\nD) for i in x:\n',
 3: '\nQuestion 3: What is the output of the following code in Python?\nmy_list = [1, 2, 3]\nmy_list.append(4)\n\nA) [1, 2, 3, 4]\nB) [4, 2, 3, 1]\nC) 4\nD) None\n',
 4: '\nQuestion 4: What is the proper syntax for declaring a function in Python?\nA) function funcName()\nB) def funcName()\nC) funcName()\nD) funcName = ()\n'}

In [23]:
def extract_answers(test, num_questions):
    answers = {1 : ""}
    question_number = 1
    for line in test.split("\n"):
        if line.startswith("Correct Answer:"):
            answers[question_number] += line+"\n"

            if question_number < num_questions:
                question_number+=1
                answers[question_number] = ""
    return answers



In [24]:
extract_answers(response["choices"][0]["text"], 4)

{1: 'Correct Answer: B) Python\n',
 2: 'Correct Answer: B) for i in range(x):\n',
 3: 'Correct Answer: A) [1, 2, 3, 4]\n',
 4: 'Correct Answer:\n'}

### Exam simulation
Based on the extracted questions, we can now simulate the exam

In [25]:
def take(student_view):
    answers = {}
    for question, question_view in student_view.items():
        print(question_view)
        answer = input("Enter your answer: ")
        answers[question] = answer
    return answers


In [26]:
student_answers = take(create_student_view(response["choices"][0]["text"], 4))



Question 1: What is the main programming language used for the Raspberry Pi?
A) Java 
B) Python 
C) HTML 
D) C++ 

Enter your answer: d

Question 2: What is the correct syntax for a for loop in Python?
A) for (i=0; i<x; i++)
B) for i in range(x):
C) for (i=0; i>x; i++)
D) for i in x:

Enter your answer: d

Question 3: What is the output of the following code in Python?
my_list = [1, 2, 3]
my_list.append(4)

A) [1, 2, 3, 4]
B) [4, 2, 3, 1]
C) 4
D) None

Enter your answer: a

Question 4: What is the proper syntax for declaring a function in Python?
A) function funcName()
B) def funcName()
C) funcName()
D) funcName = ()

Enter your answer: b


### Automatic Grading
Based on the student's answers and correct answers, we can now grade the test!

In [30]:
def grade(correct_answer_dict, answers):
    correct_answers = 0
    for question, answer in answers.items():
        if answer.upper() == correct_answer_dict[question].upper()[16]:
            correct_answers+=1
    grade = 100 * correct_answers / len(answers)

    if grade < 60:
        passed = "Not passed!"
    else:
        passed = "Passed!"
    return f"{correct_answers} out of {len(answers)} correct! You achieved: {grade} % : {passed}"


In [31]:
grade(extract_answers(response["choices"][0]["text"], 4), student_answers)

IndexError: string index out of range